In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


Eu tenho um arquivo secret.py com uma informação sigilosa. Neste caso é uma chave.
Eu vou chamar o conteudo dessa chave usando o código abaixo.


In [5]:
from secret import IEX_CLOUD_API_TOKEN

## Fazer a primeira chamada a API

A api que eu estou usando

A documentação da API que eu estou usando se encontra aqui: https://iexcloud.io/docs/api/


Eu vou buscar na API duas informações importantes:
O market capitalization para cada ação e o preço de cada ação.

In [7]:
symbol = 'AAPL'
api_url = 'https://sandbox.iexapis.com/stock/{symbol}/quote/'

### Um pouco sobre fstrem

Eu uso fstream para modificar alguma tag.

Por exemplo no código abaixo, eu vou trocar a tag {adjective} pela variável adjective acima.

In [9]:
adjective = "superb"
string = f"freeCodeCamp is {adjective}"
print(string)

freeCodeCamp is superb


Parei em 37:45